# Teams

A team is a group of agents that work together to achieve a common goal.

## RoundRobinGroupChat

A team that runs a group chat with participants taking turns in a round-robin fashion.

### Creating a Team

In [4]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_core.models import UserMessage

# client = OpenAIChatCompletionClient(
#     model=OPENAI_MODEL,
#     api_key=OPENAI_API_KEY,
#     base_url=OPENAI_API_BASE,
# )

client = OllamaChatCompletionClient(
    model=OPENAI_MODEL
)

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

def add_tool(x: float, y: float) -> float:
    """
    Add X and Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        float: The sum of X and Y.
    """
    return x + y

def subtract_tool(x: float, y: float) -> str:
    """
    Subtract Y from X to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The difference of X and Y.
    """
    return str(x - y)

def multiply_tool(x: float, y: float) -> str:
    """
    Multiply X and Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The product of X and Y.
    """
    return str(x * y)

def divide_tool(x: float, y: float) -> str:
    """
    Divide X by Y to get the exact result.

    Args:
        x (float): First number.
        y (float): Second number.

    Returns:
        str: The quotient of X and Y.
    """
    return str(x / y)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=client,
    tools=[
        add_tool,
        subtract_tool,
        multiply_tool,
        divide_tool,
    ],
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

### Run a Team

In [ ]:
result = await team.run(task="What is 1.33+2.97*3.22?")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is 1.33+2.97*3.22?', type='TextMessage'), ToolCallRequestEvent(source='primary', models_usage=RequestUsage(prompt_tokens=649, completion_tokens=29), metadata={}, content=[FunctionCall(id='1', arguments='{"x": 2.97, "y": 3.22}', name='multiply_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='primary', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='9.563400000000001', name='multiply_tool', call_id='1', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='primary', models_usage=None, metadata={}, content='9.563400000000001', type='ToolCallSummaryMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=134, completion_tokens=137), metadata={}, content='To solve the expression \\(1.33 + 2.97 \\times 3.22\\), we need to follow the order of operations (PEMDAS/BODMAS). First, perform the multiplication:\

### Observing a Team

In [11]:
await team.reset() # Reset the team for the new task.
async for message in team.run_stream(task="What is 1.33+2.97*3.22?"):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='What is 1.33+2.97*3.22?' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=547, completion_tokens=66) metadata={} content=[FunctionCall(id='2', arguments='{"x": 2.97, "y": 3.22}', name='multiply_tool'), FunctionCall(id='2', arguments='{"x": 1.33, "y": 9.5634}', name='add_tool')] type='ToolCallRequestEvent'
source='primary' models_usage=None metadata={} content=[FunctionExecutionResult(content='9.563400000000001', name='multiply_tool', call_id='2', is_error=False), FunctionExecutionResult(content='10.8934', name='add_tool', call_id='2', is_error=False)] type='ToolCallExecutionEvent'
source='primary' models_usage=None metadata={} content='9.563400000000001\n10.8934' type='ToolCallSummaryMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=75, completion_tokens=104) metadata={} content='The correct calculation for 1.33 + 2.97 * 3.22 is:\n\nFirst, multiply 2.97 by 3.22, which equals approximately 9.5

In [15]:
await team.reset()
await Console(team.run_stream(task="What is (1.33+2.97)^3.22?"))

---------- user ----------
What is (1.33+2.97)^3.22?
---------- primary ----------
[FunctionCall(id='4', arguments='{"x": 1.33, "y": 2.97}', name='add_tool')]
---------- primary ----------
[FunctionExecutionResult(content='4.300000000000001', name='add_tool', call_id='4', is_error=False)]
---------- primary ----------
4.300000000000001
---------- critic ----------
Your calculation seems a bit off. Let's break it down:

First, calculate the sum inside the parentheses: \(1.33 + 2.97 = 4.30\).

Next, raise this result to the power of 3.22: \(4.30^{3.22} \approx 86.945\).

So, the correct answer is approximately 86.945, not 4.300000000000001. Please check your exponentiation process. APPROVE once you've verified this calculation.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is (1.33+2.97)^3.22?', type='TextMessage'), ToolCallRequestEvent(source='primary', models_usage=RequestUsage(prompt_tokens=547, completion_tokens=138), metadata={}, content=[FunctionCall(id='4', arguments='{"x": 1.33, "y": 2.97}', name='add_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='primary', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='4.300000000000001', name='add_tool', call_id='4', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='primary', models_usage=None, metadata={}, content='4.300000000000001', type='ToolCallSummaryMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=67, completion_tokens=130), metadata={}, content="Your calculation seems a bit off. Let's break it down:\n\nFirst, calculate the sum inside the parentheses: \\(1.33 + 2.97 = 4.30\\).\n\nNext, raise this resul

### Stopping a Team

